# Modeling & Evaluation

Inhaltsverzeichnis

In [1]:
%load_ext autoreload
%autoreload 2

---

## Topic Modeling

### Modeling

#### A. Modeling

Der nachfolgende Code importiert für das Topic Modeling optimierte Klassen. Diese führen den Prozess der Erstellung eines LDA Topic Modeling durch. Es wird empfohlen die vordefinierten Klassen zur Berechnung der Modelle zu verwenden. Es besteht jedoch nachfolgend auch die Möglichkeit, die wichtigsten Schritte zur Berechnung des Models eigenständig Schritt für Schritt durchzuführen.

In [6]:
from src.models import topic_modeling as tm
from src.utils import safe_as_pkl
import pandas as pd

# load dataframe
df = pd.read_feather('../data/processed/twitter_tweets_processed.feather')

# create & build lda model
lda_model = tm.LdaModel(text=df['preprocessed_text'])
lda_model.build(num_topics=5)

# export
safe_as_pkl(lda_model, path='../models/lda_model.pkl')

2023-05-25 15:24:06,599 - INFO - Initialize model; create dictionary and corpus...
2023-05-25 15:24:40,090 - INFO - calculate lda model... (this can take a while)
2023-05-25 15:29:37,957 - INFO - Done. Model calculated successfully! Calculation time: 6.3369 minutes


#### 0. Datensatz laden

In [14]:
%%script false
import pandas as pd
# load dataframe
df = pd.read_feather('../data/processed/twitter_tweets_processed.feather')
df.head(4)

Couldn't find program: 'false'


#### 1. Erstellen eines Wörterbuchs

Ein Wörterbuch ist wichtig für das LDA-Modell, da es das Modell trainiert, die Sprache und den Kontext der Dokumente zu verstehen. Es enthält alle eindeutigen Wörter, die in den Dokumenten vorkommen, und ordnet jedem Wort eine eindeutige Nummer zu. Das Wörterbuch wird verwendet, um jedes Dokument in eine Vektordarstellung umzuwandeln, die das Modell verwenden kann.

In [15]:
%%script false
from gensim import corpora
dictionary = corpora.Dictionary(df['preprocessed_text'])

Couldn't find program: 'false'


#### 2. Erstellen eines Corpus

Ein Corpus ist notwendig, um jedes Dokument in eine Vektordarstellung umzuwandeln, die vom LDA-Modell verarbeitet werden kann. Ein Corpus ist eine Sammlung von Dokumenten, die in eine Matrix umgewandelt wird, wobei jede Zeile für ein Dokument und jede Spalte für ein Wort im Wörterbuch steht. Die Matrix enthält die Anzahl der Vorkommen jedes Wortes in jedem Dokument. Durch die Umwandlung jedes Dokuments in eine Vektordarstellung kann das LDA-Modell jedes Dokument analysieren und Themen identifizieren, die in jedem Dokument vorkommen. Ohne diese Vektordarstellung könnte das Modell die Dokumente nicht analysieren und Themen identifizieren.

In [16]:
%%script false
corpus = [dictionary.doc2bow(text) for text in df['preprocessed_text']]

Couldn't find program: 'false'


#### 3. Erstellen eines LDA-Modells

In [17]:
%%script false
from gensim import models
lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5)

Couldn't find program: 'false'


#### B. Ergebnisse visualisieren

In [7]:
import pyLDAvis.gensim_models
from src.utils import load_pkl

lda_model = load_pkl('../models/lda_model.pkl')

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model.model, lda_model.corpus, lda_model.dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.162041  0.144131       1        1  30.795186
4     -0.060638 -0.009188       2        1  23.105092
3     -0.164075  0.080613       3        1  22.246902
0      0.029021 -0.344181       4        1  16.752006
2      0.357732  0.128624       5        1   7.100814, topic_info=              Term           Freq          Total Category  logprob  loglift
53              ai   81781.000000   81781.000000  Default  30.0000  30.0000
853  cybersecurity   42737.000000   42737.000000  Default  29.0000  29.0000
95        technews   76808.000000   76808.000000  Default  28.0000  28.0000
50            tech  419510.000000  419510.000000  Default  27.0000  27.0000
34      innovation  215603.000000  215603.000000  Default  26.0000  26.0000
..             ...            ...            ...      ...      ...      ...
76            news    7455.672363   80244.791212   Topic5  -4.9087   0.2689
463       software    5893.605068   49293.767030   Topic5  -5.1438   0.5210
114            via    4424.798722   45740.486712   Topic5  -5.4305   0.3092
572        digital    4290.351266   40634.706645   Topic5  -5.4613   0.3967
19        business    4124.262475   78512.392155   Topic5  -5.5008  -0.3014

[321 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
2110      2  0.000256             --
2110      3  0.999609             --
868       2  0.978208  100daysofcode
868       4  0.021781  100daysofcode
235       2  0.999945             5g
...     ...       ...            ...
634       4  0.000596          would
1068      1  0.221831           year
1068      2  0.067301           year
1068      3  0.701461           year
1068      4  0.009400           year

[493 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 4, 1, 3])

### Evaluation

#### 1. Coherence Score berechnen

Der Coherence Score ist ein Evaluationsmaß für Topic Models, das versucht, die Kohärenz der gefundenen Themen zu bewerten. Die Kohärenz bezieht sich darauf, wie gut die Wörter innerhalb eines Themas zusammenpassen und ob sie eine sinnvolle Bedeutung ergeben. Ein hohes Maß an Kohärenz zeigt an, dass die Themen gut definiert und interpretierbar sind.

In [8]:
from src.models import topic_modeling as tm
from src.utils import load_pkl

lda_model = load_pkl('../models/lda_model.pkl')
coherence_score = tm.evaluate(model=lda_model.model, text=lda_model.text, dictionary=lda_model.dictionary)

2023-05-25 15:35:35,399 - INFO - calculate coherence score...
2023-05-25 15:36:17,480 - INFO - Done. Coherence score calculated successfully! Score: 0.31248594695596565


#### 2. Hyperparameter Tuning durchführen

Hyperparameter Tuning ist ein wichtiger Schritt im Machine Learning, der dazu beiträgt, das bestmögliche Modell zu finden. In diesem Fall wird das bestmögliche Modell anhand des Coherence Scores bemessen. Ziel des Hyperparameter Tuning ist es demnach, das Modell mit dem höchsten Coherence Score zu finden. Hyperparameter sind Parameter, die das Verhalten des Modells beeinflussen, aber nicht direkt von den Daten gelernt werden. Sie müssen vom Anwender festgelegt werden und können eine erhebliche Auswirkung auf die Leistung des Modells haben. Folgende Parameter sollen optimiert werden:

- num_topics
- alpha
- eta
- passes

**Random Search**

Um nun eine ausreichend gute Kombination dieser Werte zu ermitteln, wird zunächst ein Random-Search Algorithmus ausgeführt. Dieser wählt eine zufällige Kombination an Parametern aus und berechnet für diese den Coherence Score. Ziel des Algorithmus ist es, eine ausreichend gute Menge an Modellen berechnet zu haben, sodass anschließend eine Einschränkung der möglichen Paramenter-Kombinationen vorgenommen werden kann.

*Wertebereiche für Hyperparameter festlegen*

In [9]:
num_topics = [i for i in range(4, 74)]
alpha = ['symmetric', 'asymmetric'] + [round(i*0.1, 1) for i in range(1, 10)]
eta = ['symmetric', 'auto'] + [round(i*0.1, 1) for i in range(1, 10)]
passes = [i for i in range(1, 2)]

In [10]:
import itertools

combinations = list(itertools.product(num_topics, alpha, eta, passes))
list_parameter_combinations = [{'num_topics': item[0], 'alpha': item[1], 'eta': item[2], 'passes': item[3]} for item in combinations]

*Random Search Algorithmus ausführen*

Hinweis: Aufgrund einer möglichst langen Laufzeit des Random-Search Algorithmus wird empfohlen stattdessen das Skript hyperparameter_tuning_headless.py extern auf einem Server auszuführen.

In [3]:
%%script false
from src.models import hyperparameter_tuning as ht
import pandas as pd

df = ht.random_search('../data/processed/twitter_tweets_processed.feather', list_parameter_combinations, multicore=True)
df.to_feather('../data/modeling/ht_results_randomsearch.feather')

Couldn't find program: 'false'


*Ergebnisse laden und auswerten*

In [2]:
import pandas as pd
df = pd.read_feather('../data/modeling/ht_results_randomsearch.feather')
df.sort_values('coherence_score', ascending=False, inplace=True)
df.head(10)

,seed,num_topics,alpha,eta,passes,coherence_score
1,1685126019,19,0.1,0.8,1,0.44004765510898436
0,1685126019,33,0.7,0.8,1,0.2059164564261829


**Grid Search**

Nachdem der Random-Search Algorithmus durchgeführt wurde, lassen sich die Wertebereiche der Hyperparameter weiter eingrenzen. Mithilfe des Grid-Search Algorithmus werden dann alle Kombinationen von Hyperparametern berechnet.

*Wertebereich für Hyperparameter einschränken*

In [ ]:
num_topics = [i for i in range(6, 16)]
alpha = ['symmetric', 'asymmetric'] + [round(i*0.1, 1) for i in range(1, 10)]
eta = ['symmetric', 'auto'] + [round(i*0.1, 1) for i in range(1, 10)]
passes = [i for i in range(1, 2)]

In [ ]:
import itertools

combinations = list(itertools.product(num_topics, alpha, eta, passes))
list_parameter_combinations = [{'num_topics': item[0], 'alpha': item[1], 'eta': item[2], 'passes': item[3]} for item in combinations]

*Grid Search Algorithmus ausführen*

Hinweis: Da die Wahrscheinlichkeit einer langen Laufzeit hoch ist, wird auch an dieser Stelle empfohlen stattdessen das Skript hyperparameter_tuning_headless.py extern auf einem Server auszuführen.

In [2]:
%%script false
from src.models import hyperparameter_tuning as ht
import pandas as pd

df = ht.grid_search('../data/processed/twitter_tweets_processed.feather', list_parameter_combinations, multicore=True)
df.to_feather('../data/modeling/ht_results_gridsearch.feather')

Couldn't find program: 'false'


*Ergebnisse laden und auswerten*

In [4]:
import pandas as pd
df = pd.read_feather('../data/modeling/ht_results_gridsearch.feather')
df.sort_values('coherence_score', ascending=False, inplace=True)
df.head(10)

Couldn't find program: 'false'


#### 3. Erstellen optimiertes LDA-Modell 

In [13]:
pass

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.258197  0.137791       1        1  56.023458
3     -0.151347 -0.226800       2        1  17.878587
1      0.086334  0.011324       3        1   8.975768
4      0.056811  0.126159       4        1   8.441228
2      0.122286 -0.004253       5        1   5.832218
5      0.144113 -0.044221       6        1   2.848740, topic_info=            Term          Freq         Total Category  logprob  loglift
80    blockchain  19467.000000  19467.000000  Default  30.0000  30.0000
0             ai  55258.000000  55258.000000  Default  29.0000  29.0000
244     business  11557.000000  11557.000000  Default  28.0000  28.0000
339       crypto  10015.000000  10015.000000  Default  27.0000  27.0000
525          nft   9125.000000   9125.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
649    financial    158.683952    879.931913   Topic6  -5.8345   1.8454
840       update    158.910562    889.716136   Topic6  -5.8331   1.8357
244     business    222.803879  11557.249042   Topic6  -5.4952  -0.3905
2056        gold    106.720141    256.858355   Topic6  -6.2312   2.6800
4245      strong    120.049351    518.767879   Topic6  -6.1135   2.0947

[321 rows x 6 columns], token_table=       Topic      Freq Term
term                       
163        1  0.961779   's
163        2  0.036627   's
163        3  0.000161   's
163        5  0.001285   's
436        1  0.885554  ...
...      ...       ...  ...
873        3  0.000543    “
873        4  0.014663    “
30066      1  0.005163    ✦
30066      3  0.010326    ✦
30066      6  0.981008    ✦

[859 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 5, 3, 6])

---

## Time Series Analysis

### Modeling

#### 1. Beiträge einem Topic zuweisen 

In [1]:
# from src import utils
# from gensim import corpora, models
# import pandas as pd

# # load dataframe
# df = pd.read_feather('../data/processed/twitter_tweets_processed.feather')

# # Load previously trained topic model
# optimized_lda_model = utils.load_pkl('../models/optimized_lda_model.pkl')[0]
# model = optimized_lda_model['model']
# corpus = optimized_lda_model['corpus']

# # iterate over each document in the corpus and assign it the most likely topic
# topics = []
# for doc in corpus:
#     doc_topics = model.get_document_topics(doc, minimum_probability=0.1)  # select minimum probability here!
#     topics.append(max(doc_topics, key=lambda x: x[1])[0])

# # add results to the dataframe and save
# df['topic'] = topics
# df.to_feather('../models/topic_assigned_twitter_tweets.feather')

---